In [162]:
#importing necessary tools
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn.model_selection import train_test_split
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import types
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.model_selection import cross_validate
from statistics import mean
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

In [3]:
filenames = ['train', 'test', 'example_submission']
for file in filenames:
    globals()[file] = pd.read_csv("house_{}.csv".format(file))

In [4]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [112]:
modelling_df = train.copy()
predicting_df = test.copy()

In [88]:
def sorting(df, target):
    preparation_df = df.copy()
    preparation_df.fillna('none', inplace=True)
    dtype_list = preparation_df.dtypes.tolist()
    for idx, val in enumerate(dtype_list):
        dtype_list[idx] = str(val)
    categorical_variables = []
    for idx, val in enumerate(preparation_df.columns.tolist()):
        if dtype_list[idx] == 'object':
            categorical_variables.append(val)
    categorical_variables.append('MSSubClass')
    sort_columns = []
    for column in categorical_variables:
        sort_columns.append(preparation_df.groupby(column).mean().sort_values(by=target).index.tolist())        
    return sort_columns, categorical_variables

In [130]:
def preparation(df, sorted_columns_list, categorical_variables):
    preparation_df = df.copy()
    preparation_df.fillna('none', inplace=True)
    for column in categorical_variables:
        globals()['{}_list'.format(column)] = preparation_df[column].tolist()
        for idx, val in enumerate(sorted_columns_list[categorical_variables.index(column)]):
            for index, value in enumerate(globals()['{}_list'.format(column)]):
                if value == val:
                    globals()['{}_list'.format(column)][index]=idx
        preparation_df[column]=globals()['{}_list'.format(column)]
        preparation_df.replace('none', 0, inplace=True)
    return preparation_df

In [155]:
def correlation(df, target, corr_constant):
    preparation_df = df.copy()
    correlation_df = preparation_df.corr()
    features = correlation_df[abs(correlation_df[target])>corr_constant][[target]].index.tolist()
    return features[0:-1]

In [95]:
def variable_prep(prepped_df, features):
    X = prepped_df[features]
    X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
    return X

In [156]:
def train_test_prep(train_df, test_df, target, corr_constant):
    sorted_columns, categorical_variables = sorting(train_df, target)
    prepped_train_df = preparation(train_df, sorted_columns, categorical_variables)
    features = correlation(prepped_train_df, target, corr_constant)
    prepped_test_df = preparation(test_df, sorted_columns, categorical_variables)
    X_test = variable_prep(prepped_test_df, features)
    X_train = variable_prep(prepped_train_df, features)
    y_train = np.array(prepped_df[target])
    return X_train, y_train, X_test

In [186]:
constants = np.arange(0, 0.1, 0.01)
r2 = []
mse = []
for constant in constants:
    X_train, y_train, X_test = train_test_prep(modelling_df, predicting_df, 'SalePrice', constant)
    mlr = linear_model.LinearRegression()
    mlr.fit(X_train, y_train)
    cross_validate_scores = cross_validate(mlr, X_train, y_train, scoring=['r2', 'neg_root_mean_squared_error'])
    r2.append(cross_validate_scores['test_r2'].mean())
    mse.append(cross_validate_scores['test_neg_root_mean_squared_error'].mean())
best_r2 = '{}, {}'.format(str(constants[r2.index(max(r2))]), str(max(r2)))
best_mse = '{}, {}'.format(str(constants[mse.index(max(mse))]), str(max(mse)))
print(best_r2)
print(best_mse)

0.06, 0.8156715436024481
0.06, -33519.24146177073


In [190]:
models = ['mlr']
for model in models:
    X_train, y_train, X_test = train_test_prep(modelling_df, predicting_df, 'SalePrice', 0.5)
    submission_df = test[['Id']]
    globals()[model].fit(X_train, y_train)
    globals()['{}_yhat'.format(model)] = globals()[model].predict(X_test)
    submission_df['SalePrice'] = globals()['{}_yhat'.format(model)] 
    submission_df.to_csv('house_{}_submission.csv'.format(model), index = False)

<ipython-input-190-d6c159124b72>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df['SalePrice'] = globals()['{}_yhat'.format(model)]
